# Import Libraries

In [1]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install --no-cache-dir transformers sentencepiece datasets torchsummaryX nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from datasets import load_dataset
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModelForSeq2SeqLM, T5ForConditionalGeneration, T5TokenizerFast
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
import gc
import re
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from collections import Counter
from torchsummaryX import summary
from sklearn.metrics import accuracy_score
import re
import time
#******************新：借助nltl标注词性****************
import nltk
nltk.download("book")
from nltk import word_tokenize
#****************************************************
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to /root/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to /root/nltk_data...
[nltk_data]    |   Package conll2002 is already up-to-date!
[nltk_data]    | Downloading package dependency_treebank to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package dependency_treebank is already up-to-date!
[nltk_data]    | Downloadi

Device:  cpu


[nltk_data]    |   Package maxent_ne_chunker is already up-to-date!
[nltk_data]    | Downloading package universal_tagset to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package universal_tagset is already up-to-date!
[nltk_data]    | Downloading package punkt to /root/nltk_data...
[nltk_data]    |   Package punkt is already up-to-date!
[nltk_data]    | Downloading package book_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package book_grammars is already up-to-date!
[nltk_data]    | Downloading package city_database to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package city_database is already up-to-date!
[nltk_data]    | Downloading package tagsets to /root/nltk_data...
[nltk_data]    |   Package tagsets is already up-to-date!
[nltk_data]    | Downloading package panlex_swadesh to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package panlex_swadesh is already up-to-date!
[nltk_data]    | Downloading package averaged_

In [4]:
#将当前位置锚定在google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preprocessing

In [5]:
#paragraph encoder
#初始化tokenizer（使得一句话分解成token并返回token_ID）
tokenizer1 = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny", do_lower_case=True)
#初始化encoder（将token_ID根据上下文转化为向量）
encoder1 = BertModel.from_pretrained("prajjwal1/bert-tiny")

#一个演示例子
i = tokenizer1('This is a tokenization example, I love swimming', return_tensors="pt")
o = encoder1(**i, output_hidden_states=True)
embedding = o.last_hidden_state.detach().numpy()
#返回了12个token，每个token是一个长度为128的向量来表达其意
print(embedding.shape)

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(1, 12, 128)


In [6]:
#download training data
datasets = load_dataset("squad_v2")
datasets

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [7]:
#这个函数按一定比例抽取上下文context中的答案和非答案部分，返回原文相应短语的embedding，和这个短语是答案（1）还是非答案（0）
def extract_and_tokenize(context, answers, tokenizer, encoder):
    embeddings, tokenized_context, _ = tokenize_and_encode_context(context, tokenizer, encoder)
    binary_labels = generate_binary_labels(answers, tokenized_context, tokenizer)
    embeddings, binary_labels = extract_answer_token_and_nonanswer_token(embeddings, binary_labels)
    return embeddings, binary_labels

In [8]:
def tokenize_and_encode_context(context, tokenizer, encoder):
    tokenized_context = []
    embeddings = []
    for text in context:
        tokenization = tokenizer(text, return_tensors="pt", max_length = 512, 
                                 truncation = True, padding='max_length')
        context_token = tokenization['input_ids'][0].detach().numpy()
        tokenized_context.append(context_token)
        out = encoder(**tokenization, output_hidden_states=True)
        embedding = out.last_hidden_state.detach().numpy()
        assert(embedding.shape[1] == len(context_token))
        embeddings.append(embedding[0])
    return embeddings, tokenized_context, tokenization

In [9]:
def generate_binary_labels(answers, tokenized_context, tokenizer):
    binary_labels = []
    for k, answer in enumerate(answers):
        answer_texts = answer['text']
        context = tokenized_context[k]
        binary_label = np.zeros(len(context))
        for text in answer_texts:
            tokenization = tokenizer(text, return_tensors="pt")
            answer_token = tokenization['input_ids'][0].detach().numpy()[1:-1]
            answer_length = len(answer_token)
            
            for i in range(len(context) - answer_length + 1):
                if np.all(context[i:i+answer_length] == answer_token):
                    binary_label[i:i+answer_length] = 1
        binary_labels.append(binary_label)

    return binary_labels

In [10]:
def extract_answer_token_and_nonanswer_token(embeddings, binary_labels):
    single_embeddings = []
    single_labels = []
    for i, embedding in enumerate(embeddings):
        for k, token in enumerate(embedding):
            if binary_labels[i][k] == 1:
                single_embeddings.append(token)
                single_labels.append(1)
            else:
                if np.random.randint(0, 100, 1)[0] == 0:
                    single_embeddings.append(token)
                    single_labels.append(0)
      
    return single_embeddings, single_labels

In [11]:
#Extract text-answers from training data
train_dataset = datasets['train']
dev_dataset = datasets['validation']

In [12]:
#数据集过大，取前2000个样本训练
partition = 2000
train_context = train_dataset['context'][:partition]
train_answers = train_dataset['answers'][:partition]
dev_context = dev_dataset['context'][:partition//10]
dev_answers = dev_dataset['answers'][:partition//10]

In [13]:
train_x, train_y = extract_and_tokenize(train_context, train_answers, tokenizer1, encoder1)
dev_x, dev_y = extract_and_tokenize(dev_context, dev_answers, tokenizer1, encoder1)

In [14]:
#训练集中有10000个非答案短语和7700个答案短语
Counter(train_y)

Counter({1: 7708, 0: 10111})

In [15]:
#模型超参
config = {
    'batch_size': 512,
    'dropout_rate': 0.5,
    'learning_rate': 1e-1,
    'epochs': 10
}

In [16]:
#构建dataloader
train_x = torch.Tensor(train_x) 
dev_x = torch.Tensor(dev_x) 
train_y = torch.Tensor(train_y)
dev_y = torch.Tensor(dev_y) 
train_data = TensorDataset(train_x,train_y)
dev_data = TensorDataset(dev_x,dev_y)
train_dataloader = DataLoader(train_data, batch_size = config['batch_size'], 
                              shuffle = True, num_workers = 2) 
dev_dataloader = DataLoader(dev_data, batch_size = config['batch_size'], 
                              shuffle = True, num_workers = 2) 

<ipython-input-16-7809e68b0e07>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_x = torch.Tensor(train_x)


In [17]:
for x, y in train_dataloader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([512, 128])
torch.Size([512])


# Neural Answer Selector

In [18]:
#普通前馈网络结构
class Network(nn.Module):

    def __init__(self, dropout):

        super(Network, self).__init__()

        input_size = 128
        output_size = 1

        self.model = nn.Sequential(
            #layer 1
            nn.Linear(input_size, input_size * 2),
            nn.BatchNorm1d(input_size * 2),
            nn.LeakyReLU(0.2),
            

            #layer 2
            nn.Dropout(dropout),
            nn.Linear(input_size * 2, input_size * 4),
            nn.BatchNorm1d(input_size * 4),
            nn.LeakyReLU(0.2),

            #layer 3
            nn.Dropout(dropout),
            nn.Linear(input_size * 4, input_size * 8),
            nn.BatchNorm1d(input_size * 8),
            nn.LeakyReLU(0.2),


            #layer 4
            nn.Dropout(dropout),
            nn.Linear(input_size * 8, input_size * 4),
            nn.BatchNorm1d(input_size * 4),
            nn.LeakyReLU(0.2),


            #layer 5
            nn.Dropout(dropout),
            nn.Linear(input_size * 4, input_size * 2),
            nn.BatchNorm1d(input_size * 2),
            nn.LeakyReLU(0.2),

            #layer 6
            nn.Dropout(dropout),
            nn.Linear(input_size * 2, input_size), 
            nn.BatchNorm1d(input_size),
            nn.LeakyReLU(0.2),

            #layer 7
            nn.Dropout(dropout),
            nn.Linear(input_size, input_size // 2), 
            nn.BatchNorm1d(input_size // 2),
            nn.LeakyReLU(0.21),
            
            #layer 8
            nn.Dropout(dropout),
            nn.Linear(input_size // 2, input_size // 4),
            nn.BatchNorm1d(input_size // 4),   
            nn.LeakyReLU(0.2),

            #layer 9
            nn.Dropout(dropout),
            nn.Linear(input_size // 4, input_size // 8), 
            nn.BatchNorm1d(input_size // 8),  
            nn.LeakyReLU(0.2),

            #layer 10
            nn.Linear(input_size // 8, output_size),  
            nn.Sigmoid(),
        )      

    def forward(self, x):
        out = self.model(x)

        return out

In [19]:
model = Network(dropout = config['dropout_rate']).to(device)
token, label = next(iter(train_dataloader))
summary(model, token.to(device))

                        Kernel Shape Output Shape    Params Mult-Adds
Layer                                                                
0_model.Linear_0          [128, 256]   [512, 256]   33.024k   32.768k
1_model.BatchNorm1d_1          [256]   [512, 256]     512.0     256.0
2_model.LeakyReLU_2                -   [512, 256]         -         -
3_model.Dropout_3                  -   [512, 256]         -         -
4_model.Linear_4          [256, 512]   [512, 512]  131.584k  131.072k
5_model.BatchNorm1d_5          [512]   [512, 512]    1.024k     512.0
6_model.LeakyReLU_6                -   [512, 512]         -         -
7_model.Dropout_7                  -   [512, 512]         -         -
8_model.Linear_8         [512, 1024]  [512, 1024]  525.312k  524.288k
9_model.BatchNorm1d_9         [1024]  [512, 1024]    2.048k    1.024k
10_model.LeakyReLU_10              -  [512, 1024]         -         -
11_model.Dropout_11                -  [512, 1024]         -         -
12_model.Linear_12  

/usr/local/lib/python3.9/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[128, 256]","[512, 256]",33024.0,32768.0
1_model.BatchNorm1d_1,[256],"[512, 256]",512.0,256.0
2_model.LeakyReLU_2,-,"[512, 256]",NaN,NaN
3_model.Dropout_3,-,"[512, 256]",NaN,NaN
4_model.Linear_4,"[256, 512]","[512, 512]",131584.0,131072.0
5_model.BatchNorm1d_5,[512],"[512, 512]",1024.0,512.0
6_model.LeakyReLU_6,-,"[512, 512]",NaN,NaN
7_model.Dropout_7,-,"[512, 512]",NaN,NaN
8_model.Linear_8,"[512, 1024]","[512, 1024]",525312.0,524288.0


In [20]:
criterion = nn.BCELoss() #Defining Loss function 
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'max', factor = 0.7, patience = 5, verbose = True, min_lr = 1e-3)

In [21]:
torch.cuda.empty_cache()
gc.collect()

0

In [22]:
def train(model, optimizer, criterion, dataloader):

    model.train()
    train_loss = 0.0 #Monitoring Loss
    label_true_list = []
    label_pred_list = []
    
    for iter, (tokens, labels) in enumerate(tqdm(dataloader)):

        ### Move Data to Device (Ideally GPU)
        tokens = tokens.to(device)
        labels = labels.to(device)

        ### Forward Propagation
        predictions = model(tokens)

        ### Loss Calculation
        loss = criterion(predictions.reshape(-1), labels)

        train_loss += loss.item()

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()

        predictions = predictions.detach().numpy().reshape(-1)
        predictions[predictions >= 0.5] = 1
        predictions[predictions < 0.5] = 0

        ### Store Pred and True Labels
        label_pred_list.extend(list(predictions))
        label_true_list.extend(labels.detach().tolist())

        del tokens, labels, predictions
        torch.cuda.empty_cache()


    train_loss /= len(dataloader)
    accuracy = accuracy_score(label_true_list, label_pred_list) 
    return train_loss, accuracy * 100

In [23]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode

    label_true_list = []
    label_pred_list = []
    total_loss = 0.0

    for i, data in enumerate(tqdm(dataloader)):

        tokens, labels = data
        ### Move data to device (ideally GPU)
        tokens, labels = tokens.to(device), labels.to(device) 

        with torch.inference_mode(): # makes sure that there are no gradients computed as we are not training the model now
            ### Forward Propagation
            predictions = model(tokens)
            loss = criterion(predictions.reshape(-1), labels)
            total_loss += float(loss.item())
      
        predictions = predictions.cpu().numpy().reshape(-1)
        predictions[predictions >= 0.5] = 1
        predictions[predictions < 0.5] = 0
        
        
        ### Store Pred and True Labels
        label_pred_list.extend(list(predictions))
        label_true_list.extend(labels.cpu().tolist())
    
        del tokens, labels, predictions
        torch.cuda.empty_cache()

    ### Calculate Accuracy
    accuracy = accuracy_score(label_true_list, label_pred_list) 
    total_loss = float(total_loss / len(dataloader))
    return total_loss, accuracy * 100

# Experiment: Training

In [24]:
# #模型已经训练好了
# # Iterate over number of epochs to train and evaluate your model
# torch.cuda.empty_cache()

# best_accuracy = 0

# for epoch in range(config['epochs']):
#     print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

#     train_loss, train_accuracy = train(model, optimizer, criterion, train_dataloader)
#     val_loss, val_accuracy = eval(model, dev_dataloader)

#     scheduler.step(val_loss)

#     print("Train Loss: {:.4f}".format(train_loss))
#     print("Training Accuracy: {:.2f}%".format(train_accuracy))
#     print("Validation Loss: {:.4f}".format(val_loss))
#     print("Validation Accuracy: {:.2f}%".format(val_accuracy))


#     if val_accuracy > best_accuracy:
#         best_accuracy = val_accuracy
#         print("Saving model")
#         torch.save({'model_state_dict':model.state_dict(),
#                     'optimizer_state_dict':optimizer.state_dict(),
#                     'val_accuracy': val_accuracy, 
#                     'epoch': epoch}, '/content/drive/MyDrive/11611/Project/answer_extractor.pth')


# Inference

In [25]:
#Load trained model
#模型已经训练好了， 载入模型
PATH = '/content/drive/MyDrive/11611/Project/answer_extractor.pth'
QE_model = Network(dropout = config['dropout_rate']).to('cpu')
QE_model.load_state_dict(torch.load(PATH)['model_state_dict'])
QE_model.eval()

Network(
  (model): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.2)
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2)
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=1024, out_features=512, bias=True)
    (13): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): LeakyReLU(negative_slope=0.2)
    (15): Dropout(p=0.5, inplace=False)
    (16): Linear(in_features=512, out_f

In [26]:
#以下系列函数都是用来把模型预测属于答案的embeddings对应回原文相应的短语，返回一个包含答案短语字符串的[]
def predict_answer(model, context, tokenizer, encoder):
    words_list, words_ids_list, words_mapping = context_to_words_inputids(context, tokenizer)
    embeddings, _, tokenization = tokenize_and_encode_context([context], tokenizer, encoder)
    predictions = model_predict(model, embeddings)
    input_ids = tokenization['input_ids'][0].detach().numpy().tolist()
    answer_ids_list = get_answer_ids(input_ids, predictions)
    answers = map_answer_ids_to_words(answer_ids_list, words_ids_list, words_mapping, words_list)
    answers = sorted(set(answers), key=answers.index)
    sentences = split_into_sentences(context)
    answers_sentences = align_answers_with_sentences(answers, sentences)
    return answers_sentences

In [27]:
def align_answers_with_sentences(answers, sentences):
    answers_sentences = []
    for answer in answers:
        for sentence in sentences:
            if answer in sentence:
                answers_sentences.append((answer, sentence))
                break
    return answers_sentences

In [28]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [29]:
def map_answer_ids_to_words(answer_ids_list, words_ids_list, words_mapping, words_list):
    text_answer_list = []
    for ans in answer_ids_list:
        ans_length = len(ans)
        for i in range(len(words_ids_list) - ans_length + 1):
            if words_ids_list[i:i+len(ans)] == ans:
                corresponding_words = words_mapping[i:i+len(ans)]
                corresponding_words = sorted(set(corresponding_words), key=corresponding_words.index)
                text_ans = ""
                for w in corresponding_words:
                    text_ans += ' ' + words_list[w]
                text_answer_list.append(text_ans[1:])
    return text_answer_list

In [30]:
def model_predict(model, embeddings):
    embeddings = torch.tensor(embeddings[0])
    predictions = model(embeddings).flatten()
    predictions[predictions >= 0.5] = 1
    predictions[predictions < 0.5] = 0
    predictions = predictions.detach().numpy().tolist()
    return predictions

In [31]:
def get_answer_ids(input_ids, predictions):
    assert(len(input_ids) == len(predictions))
    all_answers_ids = []
    
    i = 0
    while i < len(predictions) - 1:
        if predictions[i] == 1:
            ans_span = [input_ids[i]]
            j = i + 1
            while j < len(predictions):
                if predictions[j] == 1:
                    ans_span.append(input_ids[j])
                    j += 1
                else:
                    all_answers_ids.append(ans_span)
                    i = j
                    break
        else:
            i += 1
            
    return all_answers_ids

In [32]:
def context_to_words_inputids(context,  tokenizer):
    word_ids_list = []
    word_mapping = []
    words_list = context_to_words(context)
    for k, word in enumerate(words_list):
        word_id = tokenizer(word)['input_ids'][1:-1]
        word_ids_list.extend(word_id)
        word_mapping.extend([k for i in range(len(word_id))])
    return words_list, word_ids_list, word_mapping

In [33]:
def context_to_words(context):
    return re.sub("[^\w]", " ",  context).split()

In [34]:
#since in the dataset a lot of context is same, we remove duplicates
#测试：去除测试集中的重复文章（测试集都是一个文章对好几个答案）
dev_context_no_dup = list(set(dev_context))

In [35]:
#对每篇测试文章都生生成对应的若干答案
pred_answers = []
for cont in tqdm(dev_context_no_dup):
    pred_answer = predict_answer(QE_model, cont, tokenizer1, encoder1)
    pred_answers.append(pred_answer)

100%|██████████| 38/38 [00:02<00:00, 12.98it/s]


In [36]:
# with open("/content/drive/MyDrive/11611/Project/context.txt", "w") as f:
#     for cont in dev_context_no_dup:
#         f.write(cont)
#         f.write('\n')

In [37]:
# with open("/content/drive/MyDrive/11611/Project/potential_answers.txt", "w") as f:
#     for answers in pred_answers:
#         n = len(answers)
#         for i, ans in enumerate(answers):
#             f.write(ans)
#             if i == n - 1:
#                 continue
#             f.write(',')
#         f.write('\n')

# Generate Questions 

In [38]:
#******************新：MixQG model****************
MixQG_tokenizer = AutoTokenizer.from_pretrained('Salesforce/mixqg-base')
MixQG_model = AutoModelForSeq2SeqLM.from_pretrained('Salesforce/mixqg-base')
#******************新：Answer-agnostic Model****************
answer_agnositic_model = T5ForConditionalGeneration.from_pretrained("ThomasSimonini/t5-end2end-question-generation")
answer_agnositic_tokenizer = T5TokenizerFast.from_pretrained('t5-small')

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [71]:
def Get_Questions_Main(context, N = 10, question_type = 'any', question_content = 'any'):
    if question_type not in ["any", "Wh-question", "TF-question"]:
        print("Question type should be in one of the following: Wh-question, TF-question or any!")
        return None
    if question_content not in ["any", "entity", "event", "number"]:
        print("Question content should be in one of the following: entity, event, number or any!")
        return None
    if question_type == 'any' and question_content == 'any':
        T1 = time.time()
        answers_part1 = run_answer_agnostic_model(context, N)
        T2 = time.time()
        print("Answer agnostic model consumes: ", np.round(T2 - T1, 3), " seconds!")
        answers_part2 = run_answer_aware_model(context, N = N - len(answers_part1), question_type = 'any', question_content = 'any')
        T3 = time.time()
        print("Answer aware model consumes: ", np.round(T3 - T2, 3)," seconds!")
        answers = answers_part1 + answers_part2
    else:
        answers = run_answer_aware_model(context, N = N, question_type = question_type, question_content = question_content)
    print("\n\n****************************Questions****************************")
    for q in answers:
        print(q)
    return answers

In [56]:
def run_answer_agnostic_model(context, N):
    #将原文裁剪到只剩N句话
    context_sentences = split_into_sentences(context)
    context_sentences = list(np.random.choice(context_sentences, size=N, replace=False))
    context_short = ' '.join(context_sentences)

    generator_args = {
    "max_length": 2048,
    "num_beams": 4,
    "length_penalty": 1.5,
    "no_repeat_ngram_size": 10,
    "early_stopping": True,
    }
    context = "generate questions: " + context_short + " </s>"
    input_ids = answer_agnositic_tokenizer.encode(context, return_tensors="pt")
    res = answer_agnositic_model.generate(input_ids, **generator_args)
    output = answer_agnositic_tokenizer.batch_decode(res, skip_special_tokens=True)
    output = [item.split("<sep>") for item in output]
    output = answer_agnostic_model_postprocess_output(output)
    return output


In [57]:
def answer_agnostic_model_postprocess_output(output):
    output = output[0][0]
    if output[-1] != '?':
        output = output + '?'
    output = output.split('? ')
    for i in range(len(output)):
        if output[i][-1] != '?':
            output[i] = output[i] + '?'
    return output

In [58]:
def run_answer_aware_model(context, N = 10, question_type = 'any', question_content = 'any'):
    answers_sentences = predict_answer(QE_model, context, tokenizer1, encoder1)
    answers_sentences = catagorize_answers(answers_sentences, question_content)
    if len(answers_sentences) == 0:
        print("Cannot generate required questions!")
        return None
    model_input = format_type_inputs(answers_sentences, question_type)
    model_input = sample_input(model_input, N)
    pred_quesitons = get_question(model_input)
    return pred_quesitons[:N]

In [59]:
def catagorize_answers(answers_sentences, question_content):
    cate_answers = dict()
    cate_answers['entity'] = []
    cate_answers['event'] = []
    cate_answers['number'] = []
    cate_answers['any'] = []

    for pair in answers_sentences:
        answer = pair[0]
        sentence = pair[1]
        tokens = word_tokenize(answer)
        tags = nltk.pos_tag(tokens, tagset = "universal")

        NOUN = False
        VERB = False
        NUM = None
        NUM_sentence = None

        for tag in tags:
            word = tag[0]
            POS = tag[1]

            if POS == 'NOUN':
                NOUN = True
            if POS == 'VERB':
                VERB = True
            if POS == 'NUM':
                NUM = word
                NUM_sentence = sentence
          
        if NOUN or VERB or (NUM != None):
            cate_answers['any'].append(pair)
        
        if VERB:
            cate_answers['event'].append(pair)
        elif NUM != None:
            cate_answers['number'].append((NUM, sentence))
        elif NOUN:
            cate_answers['entity'].append(pair)
            
    return cate_answers[question_content]


In [60]:
def sample_input(model_input, N):
    if len(model_input) > N:
        return list(np.random.choice(model_input, size = N, replace = False))
    else:
        return model_input

In [61]:
def get_question(model_input, max_length=32):
    input_ids = MixQG_tokenizer(model_input, return_tensors="pt", padding = True).input_ids
    generated_ids = MixQG_model.generate(input_ids, max_length=32, num_beams=4) 
    questions = MixQG_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    questions = process_questions(questions)
    return questions

In [62]:
def process_questions(questions):
    questions_m = []
    for question in questions:
        if question[-1] != '?':
            question = question[0].upper() + question[1:] + '?'
        questions_m.append(question)
    return questions_m

In [63]:
def format_type_inputs(answers_sentences, question_type):
    model_input = []
    if question_type == "Wh-question":
        for pair in answers_sentences:
            model_input.append(f"{pair[0]} \\n {pair[1]}")
    elif question_type == "TF-question":
        for pair in answers_sentences:
            model_input.append(f"{'yes'} {pair[0]} \\n {pair[1]}")
    elif question_type == "any":
        for pair in answers_sentences:
            model_input.append(f"{pair[0]} \\n {pair[1]}")
            model_input.append(f"{'yes'} {pair[0]} \\n {pair[1]}")
    return model_input

# Test Example

In [48]:
cd '/content/drive/My Drive/11611/Project/'

/content/drive/My Drive/11611/Project


In [49]:
#这是要进行测试的文本
with open('long_text.txt', 'r') as file:
    context = file.read()

In [81]:
print(len(context.split()), "words in this passage!")

4645 words in this passage!


In [83]:
#1. Get_Questions_Main是主函数，N是生成问题的个数，在question_type和question_content都为any模式下，优先选Answer-agnositc模型生成的问题，如果不够会用answer-awarer模型的问题补充
#2. 若question_type和question_content任意不为any，则只会调用answer-awarer模型，且N失效，产生多少个问题由能挖掘到多少相关答案决定（如一篇文章只有2个地方出现数字，问不出10个数字问题）
#3. question_type可以是Wh-question, TF-question or any， question_content可以是entity, event, number or any。双any就是project要求的默认模式。
#4. 函数既打印问题也返回问题列表
generated = Get_Questions_Main(context, N = 50, question_type = 'any', question_content = 'any')

Answer agnostic model consumes:  47.124  seconds!
Answer aware model consumes:  70.157  seconds!


****************************Questions****************************
What was Dempsey's first international tournament?
When did Dempsey make his first appearance with the senior team?
When did Dempsey join Tottenham Hotspur on a three-year contract?
How many Premier League goals did Dempsey score in?
What national team does Clint Dempsey play for?
How many of Dempsey's passions are hip hop?
In what league did Dempsey score a hat-trick?
Where did Dempsey grow up?
Who lived in a trailer park for most of Dempsey's childhood?
Did dempsey play for the usa on November 17?
What country did Dempsey play against in 2004?
When did Dempsey score a hat-trick in the English Premier League?
Is clint dempsey the captain of the sounders?
Did clint play soccer on the sidelines?
What team did Dempsey play for at Furman University?
Did dempsey play in the starting lineup in the mls?
Is clint dempsey born on m